In [28]:
import subprocess
import os
import cv2
import numpy as np
import json
import subprocess

In [29]:
## the function takes two arguments: the dir of the left and right videos
## it finds the time difference between the two videos and trims them to make them in sync 
## then it extracts all of the frames from the left and right
## lastly it calculates the homography between any pair
## note: be sure to include crosscorrelate.praat file, the left and right videos 
## in the same folder as the notebook

def sync(left_video, right_video):
    # extract reference(left) audio wave
    left_video =left_video
    left_wav = 'left.wav'
    command_1 = "ffmpeg -i {} -ab 160k -ac 2 -ar 44100 -vn {}".format(left_video, left_wav)
    subprocess.call(command_1, shell=True)

    # extract audio wave from the video (right video) that needs to trim
    right_video =right_video
    right_wav = 'right.wav'
    command_2 = "ffmpeg -i {} -ab 160k -ac 2 -ar 44100 -vn {}".format(right_video, right_wav)

    subprocess.call(command_2, shell=True)

    # find the offset between the refference audio wave and the second audio wave

    reference_audio= left_wav
    second_audio = right_wav
    command_3 = 'Praat crosscorrelate.praat {} {}'.format(reference_audio, second_audio)
    result = subprocess.check_output(command_3,shell=True).decode("utf-8").replace('\n','')
    result = float(result)

    # Trim the second video in order to sync the two videos 
    right_trimmed = 'right_trimmed.mov'
    command_4 = 'ffmpeg -ss {} -i {} -c copy {}'.format(result,right_video, right_trimmed)
    subprocess.call(command_4, shell=True)

    # # merge the two videos
    # sync_video ='sync.mp4'
    # command_5 = '{} -i {} -filter_complex hstack {}'.format(left_video, right_trimmed, sync_video)
    # subprocess.call(command_5, shell=True)

    ## extract the first min of left video as demo video
    t_start ='00:00:00'
    t_end = '00:01:00'

    left_cut ='/Users/likai/Desktop/new_video/demos/left_cut.mp4'
    dir1 = '/Users/likai/Desktop/new_video/demos'
    if not os.path.exists(dir1):
        os.mkdir(dir1)
    command_5 = 'ffmpeg -ss {} -i {} -to {} -c copy {}'.format(t_start, left_video,t_end, left_cut)

    subprocess.call(command_5, shell=True)                


    ## extract the first min of right video as demo video
    t_start ='00:00:00'
    t_end = '00:01:00'
    right_cut = '/Users/likai/Desktop/new_video/demos/right_cut.mp4'
    command_6 = 'ffmpeg -ss {} -i {} -to {} -c copy {}'.format(t_start, right_trimmed,t_end, right_cut)

    subprocess.call(command_6, shell=True)                


    ## extract frames from left videos
    dir2 = '/Users/likai/Desktop/new_video/left_frames'
    if not os.path.exists(dir2):
        os.mkdir(dir2)
    command_7="ffmpeg -i {} -vf fps=20 -s 480x320 /Users/likai/Desktop/new_video/left_frames/left%1d.png -hide_banner".format(left_cut)
    subprocess.call(command_7, shell=True)  



    ## extract frames from right videos
    dir3 = '/Users/likai/Desktop/new_video/right_frames'
    if not os.path.exists(dir3):
        os.mkdir(dir3)
    command_8="ffmpeg -i {} -vf fps=20 -s 480x320 /Users/likai/Desktop/new_video/right_frames/right%1d.png -hide_banner".format(right_cut)
    subprocess.call(command_8, shell=True)       

    
    
    # read json file and store the left and right cordinates in left_corlist or right_corlist
    filename = '/Users/likai/Desktop/new_video/waterpolo2.json'
    with open(filename,'r') as file:
        json_data = json.load(file)


    left_corlist = []
    right_corlist = []
    num_points= len(json_data[0]['Label'].keys())+1
    for i in range(1,num_points):
        a = []
        a =[json_data[0]['Label']['point_{}'.format(i)][0]['geometry']['x'], json_data[0]['Label']['point_{}'.format(i)][0]['geometry']['y']]
        b = [json_data[1]['Label']['point_{}'.format(i)][0]['geometry']['x'], json_data[1]['Label']['point_{}'.format(i)][0]['geometry']['y']]
        left_corlist.append(b)
        right_corlist.append(a)
        i = i +1

    ## calculate homography and h.
    ### Best homography so far 
    right_corlist = np.array([[51, 152], [135, 152], [200, 159],[233, 150], [189, 25],[222, 26],[221, 58],[153, 280],[26, 293],[118, 257]])
    left_corlist = np.array([[300, 144],[373, 141],[438, 146],[474, 137],[426, 16],[461, 14],[460, 45],[422, 259],[310, 260],[365, 236]])

    # Four points in source image
    pts_src = np.array(right_corlist)

    # Four points in destination image.
    pts_dst = np.array(left_corlist)

    # Calculate Homography
    h, status = cv2.findHomography(pts_src, pts_dst)

    for i in range(1,1201):
        # Read right image.

        im_src = cv2.imread('/Users/likai/Desktop/new_video/right_frames/right{}.png'.format(i))

        # Read left image.
        im_dst = cv2.imread('/Users/likai/Desktop/new_video/left_frames/left{}.png'.format(i))

        # Warp source image to destination based on homography
        im_out = cv2.warpPerspective(im_src, h, (im_dst.shape[0],im_dst.shape[0]))

        ## stitch image
        warpedImage = cv2.warpPerspective(im_src, h, (im_dst.shape[0]+im_src.shape[1], im_dst.shape[0]))
        warpedImage[:, 0:im_dst.shape[1], :] = im_dst
        w2,h2 = im_src.shape[:2]
        imageRightTempDim = np.float32([ [0,0], [0,w2], [h2, w2], [h2,0] ]).reshape(-1,1,2)
        imageRightDim = cv2.perspectiveTransform(imageRightTempDim, h)
        rightEdge = int(np.min(imageRightDim[2:].T[0]))
        # cv2.imshow('stitched image:',warpedImage[:, :rightEdge, :])
        ##make dir
        dir1 = '/Users/likai/Desktop/new_video/pano_im'
        if not os.path.exists(dir1):
            os.makedirs(dir1)
        cv2.imwrite('/Users/likai/Desktop/new_video/pano_im/pano{}.png'.format(i), warpedImage)


    ###Produce video based on pano
    ##make dir
    dir2 = '/Users/likai/Desktop/new_video/pano_video'
    if not os.path.exists(dir2):
        os.makedirs(dir2)
    subprocess.call(["ffmpeg","-y","-r",str(20),"-i", "/Users/likai/Desktop/new_video/pano_im/pano%1d.png","-vcodec","mpeg4", "-qscale","5", "-r", str(20), '/Users/likai/Desktop/new_video/pano_video/pano_video.mov'])   



In [30]:
sync('left.mp4','right.MOV')